---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    
#     dados=np.sum(spam_data['target']==1)
    dados=spam_data[spam_data['target']!=0]
    return (dados.shape[0]/spam_data.shape[0])*100

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    cv = CountVectorizer()
    vect = cv.fit(X_train).vocabulary_
    tokens=vect.keys()
    maximo = sorted(tokens, key=lambda word: len(word), reverse=True)
#     maximo=max(vect, key=lambda x: len(x.split()))
#     vect = cv.fit_transform(X_train)
#     tokens_and_counts = list(zip(cv.get_feature_names(), np.asarray(vect.sum(axis=0)).ravel()))
#     df = pd.DataFrame(tokens_and_counts, columns=['token', 'count'])
#     df['length'] = df.token.str.len()
#     longest= df.loc[df['length'].max(), 'token']
    
    return maximo[0]

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    predictions = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

In [8]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    feature_names = np.array(vect.get_feature_names())
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    small= list(zip(np.asarray(feature_names[sorted_tfidf_index[:20]]),sorted_tfidf_index[:20]))
    smallest=pd.DataFrame(small,columns=['name','td-idf'])
    smallest.sort_values("td-idf",inplace=True)
    smallest=smallest.set_index("name")
    smallest=smallest.T.squeeze()
    big= list(zip(np.asarray(feature_names[sorted_tfidf_index[:-21:-1]]),sorted_tfidf_index[:-21:-1]))
    bigest=pd.DataFrame(big,columns=['name','td-idf'])
    bigest.sort_values("td-idf",inplace=True,ascending=False)
    bigest=bigest.set_index("name")
    bigest=bigest.T.squeeze()
#     print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:20]]))
#     print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-21:-1]]))
    return (smallest,bigest)

In [10]:
answer_four()

(name
 aaniye           689
 athletic        1046
 chef            1676
 companion       1833
 courageous      1948
 dependable      2150
 determined      2172
 diwali          2259
 exterminator    2586
 healer          3196
 listener        3919
 mornings        4332
 organizer       4715
 pest            4887
 psychiatrist    5176
 psychologist    5178
 pudunga         5184
 stylist         6197
 sympathetic     6305
 venaam          6877
 Name: td-idf, dtype: int64, name
 yup          7328
 where        7089
 too          6598
 tick         6517
 thanx        6439
 thank        6434
 okie         4651
 ok           4646
 nite         4523
 lei          3848
 home         3293
 havent       3180
 er           2496
 done         2298
 blank        1300
 beerage      1214
 anytime       933
 anything      931
 645           533
 146tf150p     274
 Name: td-idf, dtype: int64)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    predictions = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)
    
    

In [12]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
    dados_spam=spam_data[spam_data['target']!=0]
    dados_notspam=spam_data[spam_data['target']!=1]
    dados_spam["tamanho"]=dados_spam['text'].str.len()
    dados_notspam["tamanho"]=dados_notspam['text'].str.len()
    average_len_spam=np.mean(dados_spam["tamanho"])
    average_len_notspam=np.mean(dados_notspam["tamanho"])
    return (average_len_notspam,average_len_spam)

In [14]:
answer_six()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC

def answer_seven():
#     spam_data["tamanho"]=spam_data['text'].str.len()
#     dados=add_feature(spam_data["text"],spam_data["tamanho"])
#     spam_data["tamanho"]=spam_data['text'].str.len()
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_count = add_feature(X_train_vectorized, X_train.str.len())
    X_test_vectorized=vect.transform(X_test)
    X_test_count = add_feature(X_test_vectorized, X_test.str.len())
    model = SVC(C=10000)
    model.fit(X_train_count, y_train)
    predictions = model.predict(X_test_count)
    
    return roc_auc_score(y_test, predictions)
    
    


In [17]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
def answer_eight():
#     spam_data = sum(c.isdigit() for c in spam_data["text"])
    dados_spam=spam_data[spam_data['target']!=0]
    dados_notspam=spam_data[spam_data['target']!=1]
    dados_spam["digitos"]=dados_spam["text"].str.findall((r'(\d)')).str.len()
    average_dig_spam=np.mean(dados_spam["digitos"])
    dados_notspam["digitos"]=dados_notspam["text"].str.findall((r'(\d)')).str.len()
    average_dig_notspam=np.mean(dados_notspam["digitos"])

#     for i in range(dados_spam.shape[0]):
#         dados_spam["#dig"]=len(dados_spam.iloc[i]["digitos"])
# (df == 0).astype(int).sum(axis=1)
#     for x in dados_spam["text"]:
#         dados_spam["digitos"]= sum(c.isdigit() for c in x)
#     dados_spam["digitos"]=dados_spam.count(axis='columns',numeric_only =int)
#     dados_spam["digitos"]= sum(c.isdigit() for c in dados_spam["text"])
#     dados_notspam["digitos"]= sum(c.isdigit() for c in dados_notspam["text"])
#     average_len_spam=np.mean(dados_spam["digitos"])
#     average_len_notspam=np.mean(dados_notspam["digitos"])
#     return (average_len_notspam,average_len_spam)
    return (average_dig_notspam,average_dig_spam)
  

In [19]:
answer_eight()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [20]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vect = CountVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_count =add_feature(X_train_vectorized, X_train.str.len())
    X_train_count2 = add_feature(X_train_count, X_train.str.findall((r'(\d)')).str.len())
    X_test_vectorized=vect.transform(X_test)
    X_test_count = add_feature(X_test_vectorized, X_test.str.len())
    X_test_count2 = add_feature(X_test_count, X_test.str.findall((r'(\d)')).str.len())
    model = LogisticRegression(C=100)
    model.fit(X_train_count2, y_train)
    predictions = model.predict(X_test_count2)

    return roc_auc_score(y_test, predictions)
#     return roc_auc_score(y_test, predictions)

In [21]:
answer_nine()

0.96873673666876048

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [22]:
def answer_ten():
    dados_spam=spam_data[spam_data['target']!=0]
    dados_notspam=spam_data[spam_data['target']!=1]
    dados_spam["non_word"]=dados_spam["text"].str.findall((r'(\W)')).str.len()
    average_non_word_spam=np.mean(dados_spam["non_word"])
    dados_notspam["non_word"]=dados_notspam["text"].str.findall((r'(\W)')).str.len()
    average_non_word_notspam=np.mean(dados_notspam["non_word"])

    
    return (average_non_word_notspam,average_non_word_spam)

In [23]:
answer_ten()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [37]:
def answer_eleven():
    vect = CountVectorizer(min_df=5, ngram_range=(2,5),analyzer='char_wb').fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_count =add_feature(X_train_vectorized, X_train.str.len())
    X_train_count2 = add_feature(X_train_count, X_train.str.findall((r'(\d)')).str.len())
    X_train_count3 = add_feature(X_train_count2, X_train.str.findall((r'(\W)')).str.len())
    X_test_vectorized=vect.transform(X_test)
    X_test_count = add_feature(X_test_vectorized, X_test.str.len())
    X_test_count2 = add_feature(X_test_count, X_test.str.findall((r'(\d)')).str.len())
    X_test_count3 = add_feature(X_test_count2, X_test.str.findall((r'(\W)')).str.len())
    model = LogisticRegression(C=100)
    model.fit(X_train_count3, y_train)
    predictions = model.predict(X_test_count3)
    sorted_coef_index=model.coef_[0].argsort()
    smallest=sorted_coef_index[:10]
    smallest=sorted(smallest)
    largest=sorted_coef_index[:-11:-1]
    largest=sorted(largest,reverse=True)
    return (roc_auc_score(y_test, predictions),smallest,largest)
#     return roc_auc_score(y_test, predictions)

In [38]:
answer_eleven()

(0.97885931107074342,
 [1143, 1189, 1310, 1620, 2907, 3218, 3227, 4471, 4520, 8245],
 [16315, 15949, 15855, 10911, 10589, 8804, 6160, 5158, 2893, 658])